In [21]:
#MAIN GUI
# -*- coding:utf-8 -*-
# Imports
import sys
import os
from cv2 import *
from Tkinter import * 
from Tkinter import Tk, Menu, Canvas
from PIL import Image, ImageTk
import tkFileDialog as filedialog
import pandas as pd

import numpy as np
import csv
import cv2 as cv2   #importing opevcv



#training the model with above generated csv file

#import test061_training00

data=pd.read_csv("dr_features_output_main.csv")
data_new=pd.read_csv("dr_features_output_main.csv")

predictions=data_new['count']
#data_new

threshold = 30
#data_new['pred_value'] = predictions.apply(lambda x: 1 if x > threshold else 0)
#data_new['pred_value00'] = data_new['pred_value']


features_raw = data_new[[ "count", "area"]]
predict_class = data_new['pred_values']


from sklearn.model_selection import train_test_split


np.random.seed(100)

X_train, X_test, y_train, y_test = train_test_split(features_raw, predict_class, train_size=0.80, random_state=1)

# Show the results of the split
#print "Training set has {} samples.".format(X_train.shape[0])
#print "Testing set has {} samples.".format(X_test.shape[0])


import sklearn
from sklearn import svm

C = 5
# Create SVM classification object 
svc = svm.SVC(kernel='linear',C=C,gamma=2)

svc.fit(X_train, y_train)


from sklearn.metrics import fbeta_score
predictions_test = svc.predict(X_test)
print(predictions_test)


from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, predictions_test))
print(classification_report(y_test, predictions_test))



def reading(imageSource):
    
    numOfContours=0
    imgPath = imageSource
    #raw_input("enter path/name.ext of the image")

    img = cv2.imread(os.path.join(imgPath))    #reading an image
    img_rs= cv2.resize(img, (430, 320))     #orginal image resizing
    #cv2.imshow('orginal', img_rs)

    #ROI manual specs
    x = 65
    y = 65
    w = 300
    h = 190

    imCrop = img_rs[y:y+h, x:x+w]
    #cv2.imshow("Image", imCrop)
    hsv_im = cv2.cvtColor(imCrop, cv2.COLOR_BGR2HSV)  #converting the  image to HSV

    clr_rng_img = cv2.inRange(hsv_im,(19,0,111), (255,255,255)) 
    #cv2.imshow("Binarise", clr_rng_img)

    #contour drawing test code
    image, contours, hierarchy = cv2.findContours(clr_rng_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    numOfContours = len(contours)   #number of contours

    #print("contours = ",numOfContours)
    area1 = []
    sumOfArea = 0 
    for contour in contours:
        x1,y1,w1,h1 = cv2.boundingRect(contour)
        cv2.rectangle(imCrop, (x1,y1), (x1+w1, y1+h1), (0,0,255), 1)
        area1.append(w1*h1)

        try:
            maxArea = max(area)
            #print("max area = ", maxArea)
        except:
            maxArea = 0
        try:
            area11.remove(maxArea)
        except:
            maxArea = 0

        sumOfArea = sum(area1)
        #print("sum of Area =",sumOfArea)
        #cv2.imshow("Bounding box",imCrop)
        cv2.imwrite("boundingBox.jpeg", imCrop)
    print(sumOfArea)
    print(numOfContours)
    input_data = pd.DataFrame([[numOfContours, sumOfArea]])

    print(input_data)
    #End test cod

    predictions_test1 = svc.predict(input_data)
    print(predictions_test1)

    if(predictions_test1):
        print("The patient is DIABETIC")
        p=1
    else:
        print("NOT DIABETIC")
        p=0


        


    #keyboard binding function
    k = cv2.waitKey(0);   
    if k == 27:           #wait for ESC key to exit
        cv2.destroyAllWindows();     #destroys all windows created

    
    csvTitle = [['count', 'area', 'prediction']]
    csvData = []
    
    csvData = [numOfContours, sumOfArea, p]
    
    with open('MainOutput.csv', 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(csvTitle)
        writer.writerow(csvData)
    
    numOfContours = 0
    sumOfArea = 0
    p = 0
    cavData = []







#imgname = "dataset/10_left.jpeg"
width = 430
height = 320
labels = []
# Function definitions
def deleteImage(canvas):
    canvas.delete("all")
    return


def quitProgram():
    gui.destroy()

    
# Main window
gui = Tk()
# setting the size of the window
canvas = Canvas(gui, bg="gray", width = 500, height =400)  
canvas.pack()  

# BROWSE BUTTON CODE
def browsefunc():
    global imgname
    imgname = filedialog.askopenfilename()
    pathLabel = Label(gui, text="Image path = "+str(imgname), anchor=N )
    pathLabel.pack()
    labels.append(pathLabel)
  




# Inside the main gui window
# Creating an object containing an image
# A canvas with borders that adapt to the image within it
def loadImage():
    
    width = 480
    height = 380    
    img = Image.open(imgname)
    im2 = img.resize((width, height), Image.NEAREST)  
    #width, height = img.size
    filename = ImageTk.PhotoImage(im2)
    canvas.image = filename  # <--- keep reference of your image
    canvas.create_image(200,0,anchor=N,image=filename)   
    canvas.pack()


#RUNNING PROGRAM
def run():
    reading(imgname)
    
    width = 430
    height = 320    
    img = Image.open("boundingBox.jpeg")
    im2 = img.resize((width, height), Image.NEAREST)  
    filename = ImageTk.PhotoImage(im2)
    canvas.image = filename  # <--- keep reference of your image
    canvas.create_image(200,0,anchor=N,image=filename)
    canvas.pack()

    
    outData=pd.read_csv("MainOutput.csv")
    count = int(outData['count'])
    area = int(outData['area'])
    pred = int(outData['prediction'])
  
    countLabel = Label(gui, text = "Count = "+str(count), anchor = W)
    areaLabel = Label(gui, text = "Area = "+str(area), anchor = W)
    if(pred):
        predLabel = Label(gui, text = "Result = DIABETIC", anchor = S)
    else:
        predLabel = Label(gui, text = "Result = NOT DIABETIC", anchor = S)

    countLabel.pack()
    areaLabel.pack()
    predLabel.pack()
    
   
    labels.append(countLabel)
    labels.append(areaLabel)
    labels.append(predLabel)


    
def clear():
    canvas.delete("all")
    for label in labels: label.destroy()
        
# Menu bar
menubar = Menu(gui)
# Adding a cascade to the menu bar:
filemenu = Menu(menubar, tearoff=0)
menubar.add_cascade(label="Files", menu=filemenu)
# Adding an option to browse for an image
filemenu.add_command(label="browse", command=browsefunc)
# Adding a load image button to the cascade menu "File"
filemenu.add_command(label="Load an image", command=loadImage)
# Adding a running program to the cascade menu "File"
filemenu.add_command(label="run code", command=run)
filemenu.add_separator()
filemenu.add_command(label="Clear", command=clear)
#Adding an option to quit the window
filemenu.add_command(label="Quit", command=quitProgram)
menubar.add_separator()


# Display the menu bar
gui.config(menu=menubar)
gui.title("EXTRACT TEXT FROM IMAGE")
gui.mainloop()

[1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 0 1 0
 0 1 0]
[[15  4]
 [ 1 20]]
              precision    recall  f1-score   support

           0       0.94      0.79      0.86        19
           1       0.83      0.95      0.89        21

   micro avg       0.88      0.88      0.88        40
   macro avg       0.89      0.87      0.87        40
weighted avg       0.88      0.88      0.87        40

17
8
   0   1
0  8  17
[0]
NOT DIABETIC


In [12]:
from Tkinter import *

root = Tk()
a=1
text = Text(root)
text.insert(INSERT, "Hello.....")
text.insert(END, a)
text.pack()

root.mainloop()